# 5 Custom Python Decorators For Your Projects

https://www.youtube.com/watch?v=xI4TJyd8FGk

In [2]:
# cache decorator

import time 
from functools import cache

@cache
def process_input(n):
    # do some calculation
    time.sleep(2)
    return n**2

print(process_input(5))
print(process_input(5))
print(process_input(5))
print(process_input(5))
print(process_input(5))

25
25
25
25
25


In [6]:
# taken time finder decorator

# old way 
import time 

def process_input(n):
    # do some calculation
    time.sleep(2)
    return n**2

start = time.perf_counter()
print(process_input(2))
end = time.perf_counter()
print(f"time taken: {(end - start):.4f}")

4
time taken: 2.0127


In [7]:
# decorator way
import time 

def time_logger(fun):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = fun(*args, **kwargs)
        end = time.perf_counter()
        print(f"time taken: {(end - start):.4f}")
        return result
    return wrapper

@time_logger
def process_input(n):
    # do some calculation
    time.sleep(2)
    return n**2

print(process_input(2))

time taken: 2.0117
4


In [15]:
# retry decorator

# old way
import random
import time

def error_prone_function():
    if random.random() < 0.9:
        raise ValueError('Error!')
    else:
        print("Success!")

while 1:
    try:
        error_prone_function()
        break
    except ValueError as e:
        time.sleep(1) # make sence in casae of api call


Success!


In [18]:
# decorator way
import random
import time

def retry(retries=3, exception=Exception, delay=2):
    def decorator(func):
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < retries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    print(f"Failed ({attempts}/{retries})")
                    time.sleep(delay)
            raise exception
        return wrapper
    return decorator

@retry(retries=5, exception=ValueError, delay=1)
def error_prone_function():
    if random.random() < 0.9:
        raise ValueError('Error!')
    else:
        print("Success!")

error_prone_function()

Failed (1/5)
Failed (2/5)
Failed (3/5)
Success!


In [19]:
# type checker decorator

# old way
import random
import time

def add(a: int, b: int): # Its just hint, not checking
    return a + b

print(add('10', 'hello'))

10hello


In [25]:
# decorator way

def type_check(*expected_types):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for arg, expected_type in zip(args, expected_types):
                if not isinstance(arg, expected_type):
                    raise TypeError(f"Expected {expected_type} but got {type(arg)}")
            return func(*args, **kwargs)
        return wrapper
    return decorator 

@type_check(int, int)
def add(a: int, b: int): # Its just hint, not checking
    return a + b

# print(add('10', 'hello'))
# print(add(10, 'hello'))
print(add(10, 20))

30


In [27]:
# debug decorator

def debug(func):
    def wrapper(*args, **kwargs):
        print(f"Calling function {func.__name__} with args {args} and kwargs {kwargs}")
        result = func(*args, **kwargs)
        print(f"Result was {result}")
        return result
    return wrapper

@debug
def add(a: int, b: int): # Its just hint, not checking
    return a + b

add(10, 20)

Calling function add with args (10, 20) and kwargs {}
Result was 30


30

In [31]:
# fetch limiter decorator

def rate_limiter(calls, period):
    def decorator(func):
        last_calls = []
        def wrapper(*args, **kwargs):
            nonlocal last_calls

            now = time.time()

            last_calls = [call_time for call_time in last_calls if now - call_time < period]

            if len(last_calls) > calls:
                raise RuntimeError("Rate limit exceded. Try again later.")
            
            last_calls.append(now)

            return func(*args, **kwargs)
        return wrapper
    return decorator

@rate_limiter(calls=2, period=5)
def fetch_data():
    print('Fetching data...')
    time.sleep(1)
    print('Recived data!!!')

fetch_data()
fetch_data()
fetch_data()
fetch_data()
fetch_data()
fetch_data()

Fetching data...
Recived data!!!
Fetching data...
Recived data!!!
Fetching data...
Recived data!!!


RuntimeError: Rate limit exceded. Try again later.